# MidcurveLLM Finetuning on Llama 2 using Ludwig

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip uninstall -y tensorflow --quiet
!pip install ludwig
!pip install ludwig[llm]

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

def clear_cache():
  if torch.cuda.is_available():
    model = None
    torch.cuda.empty_cache()

In [ ]:
import getpass
import locale; locale.getpreferredencoding = lambda: "UTF-8"
import logging
import os
import torch
import yaml

from ludwig.api import LudwigModel


os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("Token:")
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

In [ ]:
from google.colab import data_table; data_table.enable_dataframe_formatter()
import numpy as np; np.random.seed(123)
import pandas as pd
from datasets import DatasetDict, load_from_disk, Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/midcurvellm/midcurve_llm.csv')
df.head()

In [ ]:
instruction_tuning_llm_yaml = yaml.safe_load("""
model_type: llm
base_model: meta-llama/Llama-2-7b-hf
# meta-llama/Llama-2-7b-hf
# alexsherstinsky/Mistral-7B-v0.1-sharded
# mistralai/Mistral-7B-v0.1
# Salesforce/codet5-large

quantization:
 bits: 4

adapter:
 type: lora

prompt:
  template: |
    ### Instruction:
    You are a geometric modeling expert. You need to read 2D profile structure
    called 'Profile_brep' from json format and convert it to corresponding
    2D midcruve strucure called 'Midcurve_brep' also in json format.
    Below is an example:

    ### Input:
    {Profile_brep}

    ### Response:

input_features:
 - name: Profile_brep
   type: text

output_features:
 - name: Midcurve_brep
   type: text

trainer:
 type: finetune
 learning_rate: 0.0003
 batch_size: 1
 gradient_accumulation_steps: 8
 epochs: 3

backend:
 type: local
""")

In [ ]:
machine_translation_yaml = yaml.safe_load("""
input_features:
    -   name: Profile_brep
        type: text

output_features:
    -   name: Midcurve_brep
        type: text
        loss:
            type: sequence_softmax_cross_entropy
trainer:
    batch_size: 96
""")

In [ ]:
model_instruction_tuning = LudwigModel(config=instruction_tuning_llm_yaml)
# instruction_tuning_llm_yaml
# machine_translation_yaml
results_instruction_tuning = model_instruction_tuning.train(dataset=df)

In [ ]:
test_df = pd.DataFrame([
    {
        "Profile_brep": '{"Points": [[12.48, 0.65], [11.31, 23.12], [10.13, 45.59], [-1.18, 22.47], [23.79, 23.78]], "Lines": [[0, 1], [4, 1], [2, 1], [3, 1]], "Segments": [[0], [1], [2], [3]]}'
    },
    {
        "Profile_brep": '{"Points": [[12.48, 0.65], [11.31, 23.12], [23.79, 23.78], [-1.18, 22.47]], "Lines": [[0, 1], [1, 2], [3, 1]], "Segments": [[0], [1], [2]]}'
    }
])

In [ ]:
predictions_instruction_tuning_df, output_directory = model_instruction_tuning.predict(dataset=test_df)
shapes_brep_dict_list_strs = predictions_instruction_tuning_df["Midcurve_brep_response"].tolist()
print(shapes_brep_dict_list_strs)

In [ ]:
# import json
# shapes_brep_dict_list = json.loads(shapes_brep_dict_list_strs)
print(shapes_brep_dict_list)

In [ ]:
def plot_breps(shapes_brep_dict_list):
    for dct in shapes_brep_dict_list:
        profile_point_list = dct['Profile']
        profile_x_coords, profile_y_coords = zip(*profile_point_list)
        profile_brep = dct['Profile_brep']
        profile_segments = profile_brep["Segments"]
        profile_lines = profile_brep["Lines"]
        profile_segment_color = 'black'
        # Plot Profile segments
        for segment in profile_segments:
            for line_idx in segment:
                line = profile_lines[line_idx]
                x_segment = [profile_x_coords[i] for i in line]
                y_segment = [profile_y_coords[i] for i in line]
                plt.plot(x_segment + [x_segment[0]], y_segment + [y_segment[0]], color=profile_segment_color,
                         marker='o')

        midcurve_point_list = dct['Midcurve']
        midcurve_x_coords, midcurve_y_coords = zip(*midcurve_point_list)
        midcurve_brep = dct['Midcurve_brep']
        midcurve_segments = midcurve_brep["Segments"]
        midcurve_lines = midcurve_brep["Lines"]
        midcurve_segment_color = 'red'

        # Plot Midcurve segments
        for segment in midcurve_segments:
            for line_idx in segment:
                line = midcurve_lines[line_idx]
                x_segment = [midcurve_x_coords[i] for i in line]
                y_segment = [midcurve_y_coords[i] for i in line]
                plt.plot(x_segment + [x_segment[0]], y_segment + [y_segment[0]], color=midcurve_segment_color,
                         marker='x')

        plt.axis('equal')

In [ ]:
plot_breps(shapes_brep_dict_list)